In [1]:
#To Run jupyter notebook, you need to download xview training dataset and training labels from  https://challenge.xviewdataset.org/download-links
#after creating

In [2]:
import geojson
from datasets import Image as Im
import pyarrow as pa
import pandas as pd
from datasets import Dataset
import datasets
from PIL import Image
import numpy as np

/home/chris/.virtualenvs/pytorch101/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open('basic/xview_train_sliced_coco_with_supercategory.json') as f:
    gj = geojson.load(f)
df = pd.json_normalize([feat for feat in gj['annotations']])
df_image = pd.json_normalize([feat for feat in gj['images']])
df_cat = pd.json_normalize([feat for feat in gj['categories']])
#df = df.reset_index()
#del gj

In [4]:
df

,iscrowd,image_id,bbox,segmentation,category_id,id,area
0,0,1043,"[427, 19, 11, 10]","[[427, 19, 427, 29, 438, 29, 438, 19]]",3,27015,110
1,0,1043,"[506, 0, 6, 9]","[[506, 9, 512, 9, 512, 0, 506, 0]]",3,27016,54
2,0,1043,"[131, 234, 70, 60]","[[131, 234, 131, 294, 201, 294, 201, 234]]",34,27017,4200
3,0,1043,"[110, 139, 50, 39]","[[110, 139, 110, 178, 160, 178, 160, 139]]",34,27018,1950
4,0,1043,"[12, 71, 69, 50]","[[12, 71, 12, 121, 81, 121, 81, 71]]",34,27019,3450
...,...,...,...,...,...,...,...
4849,0,1768,"[103, 451, 333, 61]","[[436, 451, 103, 451, 103, 512, 436, 512]]",34,49738,20313
4850,0,1768,"[269, 134, 141, 117]","[[269, 134, 269, 251, 410, 251, 410, 134]]",34,49739,16497
4851,0,1768,"[190, 242, 73, 52]","[[190, 242, 190, 294, 263, 294, 263, 242]]",34,49740,3796
4852,0,1768,"[0, 287, 235, 217]","[[235, 504, 235, 287, 0, 287, 0, 504]]",34,49741,50995


In [5]:
df_image[-50:]

,height,width,id,file_name
144,512,512,1719,126_384_0_896_512.jpg
145,512,512,1720,126_768_0_1280_512.jpg
146,512,512,1721,126_1152_0_1664_512.jpg
147,512,512,1722,126_1536_0_2048_512.jpg
148,512,512,1723,126_1920_0_2432_512.jpg
149,512,512,1724,126_2304_0_2816_512.jpg
150,512,512,1725,126_2582_0_3094_512.jpg
151,512,512,1726,126_0_384_512_896.jpg
152,512,512,1727,126_384_384_896_896.jpg
153,512,512,1728,126_768_384_1280_896.jpg


In [6]:
df_cat.supercategory.value_counts()

supercategory
Engineering Vehicle    14
Maritime Vessel        10
Other                   9
Truck                   9
Railway Vehicle         6
Building                6
Fixed-wing Aircraft     3
Passenger Vehicle       3
Name: count, dtype: int64

In [7]:
df_cat['index'] =  range(0, len(df_cat))

In [8]:
df['new_cat'] = df['category_id'].apply(lambda x: df_cat[df_cat.id == x].supercategory.values[0])
df['image_id_file'] = df['image_id'].apply(lambda x: df_image[df_image.id == x].file_name.values[0])
df['list_of_cooridates'] = df['bbox'].apply(lambda x: [int(x[0]),int(x[1]),int(x[0])+int(x[2]),int(x[1]) +int(x[3])])

In [9]:
df['unique_id'] = pd.factorize(df['new_cat'])[0]

In [10]:
df.unique_id.value_counts()

unique_id
1    3555
0    1154
3     111
2      18
4      11
5       5
Name: count, dtype: int64

In [11]:
df.new_cat.value_counts()

new_cat
Building               3555
Passenger Vehicle      1154
Truck                   111
Other                    18
Engineering Vehicle      11
Maritime Vessel           5
Name: count, dtype: int64

In [12]:
df['shape'] = 0
df_final = pd.DataFrame()
LIST =[]; i = 0
for group_name, df2 in df.groupby('image_id_file'):
    #df2 = df2.drop(['level_0'],axis=1)
    df2 = df2.reset_index()
    df_append = pd.DataFrame(index=range(1),columns=['image_id','image','width','height','objects'])
    
    height, width, _ = np.array(Image.open('basic/'+str(group_name))).shape
    df_append.at[0,'image_id'] = df2['image_id_file'][0]
    df_append.at[0,'image'] = 'basic/'+str(group_name)
    df_append.at[0,'width'] = width
    df_append.at[0,'height'] = height
    df_append.at[0,'objects'] = dict({
        'id': df2['index'].tolist(),
        'area': df2['area'].tolist(),
        'bbox': df2['list_of_cooridates'].tolist(),
        'category': df2['unique_id'].tolist()
    }
    )
    
    LIST.append(df_append)
    if len(LIST) > 20:
        df_concat = pd.concat(LIST)
        df_final = pd.concat([df_final,df_concat])
        LIST = []
        print('finished with ' + str(i))
    i += 1
if len(LIST) > 0:
    df_concat = pd.concat(LIST)
    df_final = pd.concat([df_final,df_concat])
    
df = df_final.reset_index()
del df_final, df2,df_concat, df_append

finished with 20
finished with 41
finished with 62
finished with 83


In [13]:
hg_dataset = Dataset.from_pandas(df)
NewDataset = datasets.DatasetDict({"train":hg_dataset})#,"val":test_dataset})
newdata = NewDataset['train'].cast_column("image", Im())
NewDataset = datasets.DatasetDict({"train":newdata})#,"val":test_dataset['train']})
del newdata, hg_dataset

In [14]:
NewDataset['train'].data.nbytes / 1e9

0.000300263

In [15]:
NewDataset

DatasetDict({
    train: Dataset({
        features: ['index', 'image_id', 'image', 'width', 'height', 'objects'],
        num_rows: 84
    })
})

In [16]:
NewDataset['train'][0]

{'index': 0,
 'image_id': '125_0_1536_512_2048.jpg',
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512>,
 'width': 512,
 'height': 512,
 'objects': {'area': [1005],
  'bbox': [[497, 423, 512, 490]],
  'category': [1],
  'id': [4136]}}

In [17]:
#NewDataset.push_to_hub('Honaker/xview_dataset_subset',  num_shards={'train': 50, 'test': 0})